<a href="https://colab.research.google.com/github/nataliamarinn/dm_grupo4/blob/main/NM_pruebas_nuestras_caras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>